In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense,LSTM,BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
df = pd.read_csv('reg_ready.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [3]:
REL_VARS = ['YIELD_CURVE','KO_diff_from_2_month_high',
       'KR_diff_from_2_month_high',
       'PPG_diff_from_2_month_high', 'SHW_diff_from_2_month_high',
       'unemployment_diff_from_6_month_low', 'INDPRO_diff_from_6_month_high']



df.dropna(subset=REL_VARS,inplace=True)
df.reset_index(inplace=True)
df.head()


,index,Date,Open,Close,monthly_return,period of downturn,Recessions,INDPRO,yc_date,YIELD_CURVE,UNEMPLOYMENT,BGI_diff_from_2_month_high,KO_diff_from_2_month_high,KR_diff_from_2_month_high,MAR_diff_from_2_month_high,PPG_diff_from_2_month_high,SHW_diff_from_2_month_high,unemployment_diff_from_6_month_low,INDPRO_diff_from_6_month_high
0,362,1980-04-01,102.089996,106.290001,0.041140,0,1,53.5053,1980-04-01,-1.90,6.3,NaN,-0.041825,-0.053846,NaN,0.000000,0.000000,0.4,0.0000
1,363,1980-05-01,106.290001,111.239998,0.046571,0,1,53.3294,1980-05-01,-0.13,6.3,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.4,0.1759
2,364,1980-06-01,111.239998,114.239998,0.026969,0,1,52.2336,1980-06-02,0.97,6.9,NaN,0.000000,0.000000,NaN,0.000000,-0.068085,1.0,1.2717
3,365,1980-07-01,114.239998,121.669998,0.065039,0,1,50.9638,1980-07-01,1.12,7.5,NaN,-0.022222,0.000000,NaN,-0.008264,0.000000,1.5,2.5415
4,366,1980-08-01,121.669998,122.379997,0.005835,0,1,50.3348,1980-08-01,0.98,7.6,NaN,0.000000,0.000000,NaN,0.000000,0.000000,1.3,3.1705


#### Create windows of proper dimension (samples X timesteps X features)

In [4]:
X = []
y_downturn = []
y_monthly_return = []

for idx,dt in enumerate(df['Date']):
    obs_prior = df[df['Date']<=dt].shape[0]
    if obs_prior>=12:
        sample = []
        df_last_12 = df[df['Date']<=dt].iloc[-12:]
        sample.append(list(df_last_12['YIELD_CURVE']))
        sample.append(list(df_last_12['KO_diff_from_2_month_high']))
        sample.append(list(df_last_12['KR_diff_from_2_month_high']))
        sample.append(list(df_last_12['PPG_diff_from_2_month_high']))
        sample.append(list(df_last_12['SHW_diff_from_2_month_high']))
        sample.append(list(df_last_12['unemployment_diff_from_6_month_low']))
        sample.append(list(df_last_12['INDPRO_diff_from_6_month_high']))
        sample = np.array(sample).T
        
        X.append(sample)
        y_downturn.append(df.iloc[idx]['period of downturn'])
        y_monthly_return.append(df.iloc[idx]['monthly_return'])

In [5]:
arr1 = np.array(X)
arr1.shape

(462, 12, 7)

In [6]:
df_train = df.iloc[11:296]
print(df_train.shape)
df_test = df.iloc[297:]
print(df_test.shape)

(285, 19)
(176, 19)


#### Start with predicting monthly return on 1980 features

In [7]:
y = np.array(y_monthly_return)
X = arr1.copy()

In [8]:
trainX = X[:285,:,:]
trainY = y[:285]

testX = X[286:,:,:]
testY = y[286:]
print(trainX.shape)
print(testX.shape)  ##We can see that the dimensions of the numpy arrays are what we expected for merging with dfs.

(285, 12, 7)
(176, 12, 7)


In [9]:
# create and fit the LSTM network
batch_size = 1
features=7
timesteps=12



model = Sequential()
#model.add(LSTM(4, batch_input_shape=(batch_size, timesteps, features), return_sequences=True,activation='relu'))
#model.add(BatchNormalization())
model.add(LSTM(16, batch_input_shape=(batch_size, timesteps, features), activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

W0814 14:22:58.066321 140737112867776 deprecation_wrapper.py:119] From /Users/samuelholden/anaconda3/envs/dnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 14:22:58.182993 140737112867776 deprecation_wrapper.py:119] From /Users/samuelholden/anaconda3/envs/dnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 14:22:58.213577 140737112867776 deprecation_wrapper.py:119] From /Users/samuelholden/anaconda3/envs/dnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0814 14:22:58.949818 140737112867776 deprecation_wrapper.py:119] From /Users/samuelholden/anaconda3/envs/dnn/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprec

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, 16)                   1536      
_________________________________________________________________
dense_1 (Dense)              (1, 1)                    17        
Total params: 1,553
Trainable params: 1,553
Non-trainable params: 0
_________________________________________________________________


In [11]:
results = model.fit(trainX, trainY, epochs=15, batch_size=batch_size, verbose=2, shuffle=False,validation_data=[testX,testY])


W0814 14:22:59.146584 140737112867776 deprecation.py:323] From /Users/samuelholden/anaconda3/envs/dnn/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0814 14:22:59.423761 140737112867776 deprecation_wrapper.py:119] From /Users/samuelholden/anaconda3/envs/dnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0814 14:22:59.493442 140737112867776 deprecation_wrapper.py:119] From /Users/samuelholden/anaconda3/envs/dnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 285 samples, validate on 176 samples
Epoch 1/15
 - 2s - loss: 0.0062 - val_loss: 0.0642
Epoch 2/15
 - 2s - loss: 0.0024 - val_loss: 0.0627
Epoch 3/15
 - 2s - loss: 0.0021 - val_loss: 0.0467
Epoch 4/15
 - 2s - loss: 0.0020 - val_loss: 0.0441
Epoch 5/15
 - 2s - loss: 0.0020 - val_loss: 0.0387
Epoch 6/15
 - 2s - loss: 0.0019 - val_loss: 0.0364
Epoch 7/15
 - 2s - loss: 0.0019 - val_loss: 0.0314
Epoch 8/15
 - 2s - loss: 0.0019 - val_loss: 0.0297
Epoch 9/15
 - 2s - loss: 0.0019 - val_loss: 0.0251
Epoch 10/15
 - 2s - loss: 0.0019 - val_loss: 0.0211
Epoch 11/15
 - 2s - loss: 0.0019 - val_loss: 0.0208
Epoch 12/15
 - 2s - loss: 0.0019 - val_loss: 0.0173
Epoch 13/15
 - 2s - loss: 0.0018 - val_loss: 0.0155
Epoch 14/15
 - 2s - loss: 0.0018 - val_loss: 0.0143
Epoch 15/15
 - 2s - loss: 0.0018 - val_loss: 0.0129


In [12]:
trainPredict = model.predict(trainX, batch_size=batch_size)

In [13]:
df_train['pred'] = trainPredict
df_train.to_csv('trainset_LSTM_1980.csv',index=False)

/Users/samuelholden/anaconda3/envs/dnn/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
testPredict = model.predict(testX, batch_size=batch_size)

In [15]:
df_test['pred'] = testPredict
df_test.to_csv('testset_LSTM_1980.csv',index=False)

/Users/samuelholden/anaconda3/envs/dnn/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### Let us try with downturn classification

In [16]:
y = np.array(y_downturn)
X = arr1.copy()

trainX = X[:285,:,:]
trainY = y[:285]

testX = X[286:,:,:]
testY = y[286:]
print(trainX.shape)
print(testX.shape)  ##We can see that the dimensions of the numpy arrays are what we expected for merging with dfs.

# create and fit the LSTM network
batch_size = 1
features=7
timesteps=12

model = Sequential()
#model.add(LSTM(4, batch_input_shape=(batch_size, timesteps, features), return_sequences=True,activation='relu'))
#model.add(BatchNormalization())
model.add(LSTM(16, batch_input_shape=(batch_size, timesteps, features), activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

(285, 12, 7)
(176, 12, 7)


In [17]:
results = model.fit(trainX, trainY, epochs=7, batch_size=batch_size, verbose=2, shuffle=False,validation_data=[testX,testY])

Train on 285 samples, validate on 176 samples
Epoch 1/7
 - 2s - loss: 0.7739 - val_loss: 1.3241
Epoch 2/7
 - 2s - loss: 0.9152 - val_loss: 1.3070
Epoch 3/7
 - 2s - loss: 0.9136 - val_loss: 1.3098
Epoch 4/7
 - 2s - loss: 0.5726 - val_loss: 0.4497
Epoch 5/7
 - 2s - loss: 0.4279 - val_loss: 0.4309
Epoch 6/7
 - 2s - loss: 0.3523 - val_loss: 0.6802
Epoch 7/7
 - 2s - loss: 0.2945 - val_loss: 0.6741


In [18]:
testPredict = model.predict(testX, batch_size=batch_size)

In [19]:
df_test['pred'] = testPredict
#df_test.to_csv('testset_LSTM_classification_1980.csv',index=False)

/Users/samuelholden/anaconda3/envs/dnn/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### Let's try with fewer features - 1980, drop KO,KR, and monthly_return predicted

In [20]:
X = []
y_downturn = []
y_monthly_return = []

for idx,dt in enumerate(df['Date']):
    obs_prior = df[df['Date']<=dt].shape[0]
    if obs_prior>=12:
        sample = []
        df_last_12 = df[df['Date']<=dt].iloc[-12:]
        sample.append(list(df_last_12['YIELD_CURVE']))
        #sample.append(list(df_last_12['KO_diff_from_2_month_high']))
        #sample.append(list(df_last_12['KR_diff_from_2_month_high']))
        sample.append(list(df_last_12['PPG_diff_from_2_month_high']))
        sample.append(list(df_last_12['SHW_diff_from_2_month_high']))
        sample.append(list(df_last_12['unemployment_diff_from_6_month_low']))
        sample.append(list(df_last_12['INDPRO_diff_from_6_month_high']))
        sample = np.array(sample).T
        
        X.append(sample)
        y_downturn.append(df.iloc[idx]['period of downturn'])
        y_monthly_return.append(df.iloc[idx]['monthly_return'])
        
arr1 = np.array(X)
arr1.shape

df_train = df.iloc[11:296]
print(df_train.shape)
df_test = df.iloc[297:]
print(df_test.shape)

y = np.array(y_monthly_return)
X = arr1.copy()

(285, 19)
(176, 19)


In [21]:
trainX = X[:285,:,:]
trainY = y[:285]

testX = X[286:,:,:]
testY = y[286:]
print(trainX.shape)
print(testX.shape)  ##We can see that the dimensions of the numpy arrays are what we expected for merging with dfs.

(285, 12, 5)
(176, 12, 5)


In [22]:
# create and fit the LSTM network
batch_size = 1
features=5
timesteps=12



model = Sequential()

model.add(LSTM(16, batch_input_shape=(batch_size, timesteps, features), activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [23]:
results = model.fit(trainX, trainY, epochs=15, batch_size=batch_size, verbose=2, shuffle=False,validation_data=[testX,testY])

#trainPredict = model.predict(trainX, batch_size=batch_size)
#df_train['pred'] = trainPredict
#df_train.to_csv('trainset_LSTM_1980_reducedfeats.csv',index=False)
#
#testPredict = model.predict(testX, batch_size=batch_size)
#df_test['pred'] = testPredict
#df_test.to_csv('testset_LSTM_1980_reducedfeats.csv',index=False)

Train on 285 samples, validate on 176 samples
Epoch 1/15
 - 2s - loss: 0.0027 - val_loss: 0.0077
Epoch 2/15
 - 2s - loss: 0.0023 - val_loss: 0.0017
Epoch 3/15
 - 2s - loss: 0.0021 - val_loss: 0.0019
Epoch 4/15
 - 2s - loss: 0.0020 - val_loss: 0.0020
Epoch 5/15
 - 2s - loss: 0.0020 - val_loss: 0.0021
Epoch 6/15
 - 2s - loss: 0.0020 - val_loss: 0.0023
Epoch 7/15
 - 2s - loss: 0.0019 - val_loss: 0.0023
Epoch 8/15
 - 2s - loss: 0.0019 - val_loss: 0.0024
Epoch 9/15
 - 2s - loss: 0.0019 - val_loss: 0.0024
Epoch 10/15
 - 2s - loss: 0.0019 - val_loss: 0.0026
Epoch 11/15
 - 2s - loss: 0.0019 - val_loss: 0.0026
Epoch 12/15
 - 2s - loss: 0.0019 - val_loss: 0.0029
Epoch 13/15
 - 2s - loss: 0.0019 - val_loss: 0.0025
Epoch 14/15
 - 2s - loss: 0.0019 - val_loss: 0.0026
Epoch 15/15
 - 2s - loss: 0.0018 - val_loss: 0.0025


#### So the 1980 full features is the best (or equivalent) of the RNN models tried. Let's comment out the extra to simplify file mgmt.

#### Not better (although more sensitive) than classification RF